## Graph Adversarial Attack Evaluation
-----------------------------------
This script evaluates the performance of a Graph search before and after applying adversarial attacks using the Attack method on a dataset.

Steps:
1. Load the Cora dataset.
2. Prepare a search set with queries.
3. Initialize the GCN search method and attack method.
4. Perform adversarial attacks on selected nodes.
5. Evaluate the performance of GCN before and after the attack.
6. Save evaluation results to CSV, JSON, and Pickle formats.

In [1]:
import os
import torch
from roksana.search_methods import get_search_method
from roksana.datasets import load_dataset, prepare_search_set
from roksana.evaluation import Evaluator, save_results_to_pickle, save_results_to_json
from roksana.attack_methods import get_attack_method
from roksana.utils import *


## Main function to execute the adversarial attack evaluation workflow.

Steps:
- Load the dataset.
- Prepare a search set for evaluation.
- Initialize the GCN search method and VikingAttack.
- Perform attacks on query nodes and apply the changes to the dataset.
- Evaluate GCN performance before and after the attack.
- Save evaluation results in multiple formats.

In [2]:
# Set CUDA_LAUNCH_BLOCKING for debugging
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [3]:
# Set Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# Load the Cora dataset
dataset = load_dataset(dataset_name='cora', root='data/')
data = dataset[0]

In [5]:
# Prepare the test set with 10% of nodes as queries
queries, query_features, gold_sets = prepare_search_set(data, percentage=0.1, seed=123)

In [6]:
# Initialize the GCN search method before attack
gcn_before = get_search_method('gcn', data=data, hidden_channels=64, epochs=200, lr=0.01)

GCN Epoch: 001, Loss: 4.1796, Train Acc: 0.6143
GCN Epoch: 010, Loss: 0.5383, Train Acc: 0.9786
GCN Epoch: 020, Loss: 0.0429, Train Acc: 1.0000
GCN Epoch: 030, Loss: 0.0064, Train Acc: 1.0000
GCN Epoch: 040, Loss: 0.0021, Train Acc: 1.0000
GCN Epoch: 050, Loss: 0.0011, Train Acc: 1.0000
GCN Epoch: 060, Loss: 0.0008, Train Acc: 1.0000
GCN Epoch: 070, Loss: 0.0007, Train Acc: 1.0000
GCN Epoch: 080, Loss: 0.0006, Train Acc: 1.0000
GCN Epoch: 090, Loss: 0.0005, Train Acc: 1.0000
GCN Epoch: 100, Loss: 0.0005, Train Acc: 1.0000
GCN Epoch: 110, Loss: 0.0005, Train Acc: 1.0000
GCN Epoch: 120, Loss: 0.0004, Train Acc: 1.0000
GCN Epoch: 130, Loss: 0.0004, Train Acc: 1.0000
GCN Epoch: 140, Loss: 0.0004, Train Acc: 1.0000
GCN Epoch: 150, Loss: 0.0004, Train Acc: 1.0000
GCN Epoch: 160, Loss: 0.0003, Train Acc: 1.0000
GCN Epoch: 170, Loss: 0.0003, Train Acc: 1.0000
GCN Epoch: 180, Loss: 0.0003, Train Acc: 1.0000
GCN Epoch: 190, Loss: 0.0003, Train Acc: 1.0000
GCN Epoch: 200, Loss: 0.0003, Train Acc:

In [7]:
# Initialize the attack method
attack_method = get_attack_method('random', data=data, perturbations=2, device=device)

In [8]:
# Perform attacks on all query nodes
removed_edges_list = []
for query in queries:
    tensor_query = torch.tensor(query, device=device)
    updated_data, removed_edge = attack_method.attack(data=data, selected_nodes=tensor_query)
    removed_edges_list.append(removed_edge)
removed_edges_list_stat(data, removed_edges_list)

Number of Removed Edges in the list: 270
Number of Duplicate Edges: 2
Number of Unique Edges to Remove: 268
Number of Removed Edges with Reversed Counterparts in Graph: 268
Number of Total Removed Edges: 536


In [9]:
# Apply all removed edges to the final data 
# for edge in removed_edges_list:
attacked_data = remove_edges(data, removed_edges_list, inplace=False)

In [10]:
compare_original_vs_updated(data, attacked_data)

____________________________________________
- Compare
____________________________________________
Number of nodes (Original): 2708
Number of nodes (Updated): 2708
Number of edges (Original): 10556
Number of edges (Updated): 10020

Difference in the number of edges: 536
____________________________________________
____________________________________________


In [11]:
# Initialize the GCN search method after attack
gcn_after = get_search_method('gcn', data=attacked_data, hidden_channels=64, epochs=200, lr=0.01)

GCN Epoch: 001, Loss: 4.1785, Train Acc: 0.6214
GCN Epoch: 010, Loss: 0.6218, Train Acc: 0.9643
GCN Epoch: 020, Loss: 0.0627, Train Acc: 1.0000
GCN Epoch: 030, Loss: 0.0090, Train Acc: 1.0000
GCN Epoch: 040, Loss: 0.0028, Train Acc: 1.0000
GCN Epoch: 050, Loss: 0.0015, Train Acc: 1.0000
GCN Epoch: 060, Loss: 0.0010, Train Acc: 1.0000
GCN Epoch: 070, Loss: 0.0008, Train Acc: 1.0000
GCN Epoch: 080, Loss: 0.0007, Train Acc: 1.0000
GCN Epoch: 090, Loss: 0.0006, Train Acc: 1.0000
GCN Epoch: 100, Loss: 0.0006, Train Acc: 1.0000
GCN Epoch: 110, Loss: 0.0005, Train Acc: 1.0000
GCN Epoch: 120, Loss: 0.0005, Train Acc: 1.0000
GCN Epoch: 130, Loss: 0.0005, Train Acc: 1.0000
GCN Epoch: 140, Loss: 0.0004, Train Acc: 1.0000
GCN Epoch: 150, Loss: 0.0004, Train Acc: 1.0000
GCN Epoch: 160, Loss: 0.0004, Train Acc: 1.0000
GCN Epoch: 170, Loss: 0.0003, Train Acc: 1.0000
GCN Epoch: 180, Loss: 0.0003, Train Acc: 1.0000
GCN Epoch: 190, Loss: 0.0003, Train Acc: 1.0000
GCN Epoch: 200, Loss: 0.0003, Train Acc:

In [12]:
# Initialize the Evaluator
evaluator = Evaluator(
    search_method_before=gcn_before,
    search_method_after=gcn_after,
    k_values=[5, 10, 20]
)

In [13]:
# Perform evaluation and save results to CSV
evaluator.evaluate(
    queries=torch.tensor(queries).to(device),
    gold_sets=gold_sets,
    results_dir='evaluation_results',
    filename='gcn_attack_evaluation.csv'
)

Evaluation results saved to evaluation_results/gcn_attack_evaluation.csv


In [14]:
# Optionally, save results to JSON or Pickle
results = evaluator.get_all_results()  # Assuming you have a method to retrieve all results
save_results_to_json(results, 'evaluation_results/gcn_attack_evaluation.json')
save_results_to_pickle(results, 'evaluation_results/gcn_attack_evaluation.pkl')

Results successfully saved to evaluation_results/gcn_attack_evaluation.json in JSON format.
Results successfully saved to evaluation_results/gcn_attack_evaluation.pkl in Pickle format.
